<a href="https://colab.research.google.com/github/ronyarmon/AMR/blob/master/query_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Functions

In [1]:
import os
import pandas as pd 
from google.colab import drive
from sqlite3 import connect
conn = connect(':memory:')  

#Columns metadata

In [2]:
data_results_path = '/content/drive/MyDrive/data_results/mlb-player-digital-engagement-forecasting'
drive.mount('/content/drive')
file_path = os.path.join(data_results_path, 'columns_classification.csv')
columns_md = pd.read_csv(file_path)
predictors = list(columns_md['column'][columns_md['role']=='predictor'])
date_fields = list(columns_md['column'][columns_md['role']=='date'])

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
columns_md.head()

,column,topic,role,source_type,inferred_type,values_count
0,gameTimeUTC,games,time,object,numeric,5108
1,gameTimeUTC,teamBoxScores,time,object,numeric,4501
2,gameTimeUTC,events,time,object,numeric,4499
3,gamePk,games,predictor,int64,numeric,7693
4,season,events,predictor,int64,ordered,4


#Build Topics and Target Databases

In [ ]:
data_files = os.listdir(data_results_path)
data_files = [tf for tf in data_files if tf!='columns_classification.csv']
for data_file in data_files:
  print ('data file:', data_file)
  data_path = os.path.join(data_results_path, '{t}'.format(t=data_file))
  data_file_db_name = data_file.replace ('.csv', '')
  print ('data_file_db_name:', data_file_db_name)
  pd.read_csv(data_path).to_sql('{df}'.format(df=data_file_db_name), conn)

#Topics with predicate fields

##ID columns

In [36]:
id_columns = list (columns_md['column'][columns_md['role']=='id'])
print('id_columns:', id_columns)
print ('id columns in more than one topic')
id_columns_topics = {}
for id_column in id_columns:
  id_column_topics = list(columns_md['topic'][columns_md['column']==id_column])
  #print (id_column, id_column_topics)
  if len (id_column_topics)>1:
    id_columns_topics [id_column] = id_column_topics

for k,v in id_columns_topics.items(): print (k,v)

id_columns: ['playerId', 'awayId', 'pitcherTeamId', 'playerId', 'teamId', 'eventId', 'homeId', 'teamId', 'toTeamId', 'fromTeamId', 'awardId', 'teamId', 'divisionId', 'transactionId', 'playId', 'hitterTeamId', 'pitcherId', 'playerId', 'playerId', 'teamId', 'awardPlayerTeamId', 'hitterId']
topics with id columns
playerId ['transactions', 'playerTwitterFollowers', 'awards', 'rosters']
teamId ['standings', 'teamBoxScores', 'teamTwitterFollowers', 'rosters']


## Date columns

In [31]:
date_columns = list (columns_md['column'][columns_md['role']=='date'])
print ('date_columns:', date_columns)
playerId_topics = list (columns_md['topic'][columns_md['column']=='playerId'])
print('playerId_topics:', player_id_topics)
# playerId_date_topics 
topics = list(set(columns_md['topic']))
print ('topics:', topics)
playerId_date_topics = {}
for topic in topics:
  topic_columns = list(columns_md['column'][columns_md['topic']==topic])
  print(topic, topic_columns)
  topic_date_columns = [c for c in topic_columns if c in date_columns]
  print (topic_date_columns)
  print (30*'-')

id_columns: ['playerId', 'awayId', 'pitcherTeamId', 'playerId', 'teamId', 'eventId', 'homeId', 'teamId', 'toTeamId', 'fromTeamId', 'awardId', 'teamId', 'divisionId', 'transactionId', 'playId', 'hitterTeamId', 'pitcherId', 'playerId', 'playerId', 'teamId', 'awardPlayerTeamId', 'hitterId']
topics with id columns
playerId ['transactions', 'playerTwitterFollowers', 'awards', 'rosters']
awayId ['games']
pitcherTeamId ['events']
playerId ['transactions', 'playerTwitterFollowers', 'awards', 'rosters']
teamId ['standings', 'teamBoxScores', 'teamTwitterFollowers', 'rosters']
eventId ['events']
homeId ['games']
teamId ['standings', 'teamBoxScores', 'teamTwitterFollowers', 'rosters']
toTeamId ['transactions']
fromTeamId ['transactions']
awardId ['awards']
teamId ['standings', 'teamBoxScores', 'teamTwitterFollowers', 'rosters']
divisionId ['standings']
transactionId ['transactions']
playId ['events']
hitterTeamId ['events']
pitcherId ['events']
playerId ['transactions', 'playerTwitterFollowers', '


#Featue Join Plan

Goal: Join data per feature to target on playerIds and dates
pId_df = playerId_dataset
d_df = dates_dataset
features = columns in piD_df
each piD_df may include more than one date columns
dcols = date columns in piD_df (dcol 1..n dcols; max(n) appears 2-3)
for each dcol dcols:

    feature_df = piD_df (feature, dcol, playerID)
    join feature_df to target_df on:
    feature_df.playerID = target_df.playerID
    feature_df.dcol = target_df.date



#Query Predictors

In [23]:
for predictor in predictors:
  print ('predictor:', predictor)
  topic = columns_md['topic'][columns_md['column']==predictor].values[0]
  print (predictor, topic)
  topic_path = os.path.join(data_results_path, '{t}.csv'.format(t=topic))
  topic_df = pd.read_csv(topic_path)
  print (topic_df.head())
  topic_columns = topic_df.columns
  date_columns = [t for t in topic_columns if t in date_columns]
  print ('date_columns:', date_columns)
  if ((len(date_columns)>0) & ('playerId' in topic_columns)):
    for date_column in date_columns:
      query = """SELECT {to}.{p}, {ta}.target1 
                FROM {to} JOIN {ta} 
                on to.playerId  = ta.playerId"""\
                .format(to=topic, ta=target_df, p=predictor)
      print (query)
      query_df = pd.read_sql(query, conn)
      print (type (query_df))
      print(query_df.head())
      print(query_df.info())
      

predictor: gamePk
gamePk games
   gamePk gameType  season  ... awayWinPct awayWinner awayScore
0  533782        E    2018  ...        0.0        0.0       2.0
1  534461        E    2018  ...        0.0        0.0       1.0
2  545334        E    2018  ...        0.0        0.0       0.0
3  547295        E    2018  ...        0.0        0.0       2.0
4  533784        E    2018  ...        0.0        0.0       1.0

[5 rows x 32 columns]
date_columns: ['gameDate', 'resumeDate']
predictor: season
season events
   gamePk    gameDate           gameTimeUTC  season  ...  y0  z0    type  zone
0  529409  2018-03-29  2018-03-30T02:10:00Z    2018  ... NaN NaN  action   NaN
1  529416  2018-03-29  2018-03-29T20:10:00Z    2018  ... NaN NaN  action   NaN
2  529416  2018-03-29  2018-03-29T20:10:00Z    2018  ... NaN NaN  action   NaN
3  529411  2018-03-29  2018-03-29T19:35:00Z    2018  ... NaN NaN  action   NaN
4  529406  2018-03-29  2018-03-29T20:00:00Z    2018  ... NaN NaN  action   NaN

[5 rows x 71 c

KeyboardInterrupt: ignored

In [ ]:
# Query Model
dfa.to_sql('test_a', conn)
dfb.to_sql('test_b', conn)

query = """SELECT test_a.name as namea,b,c,d,e 
           FROM test_a JOIN test_b 
           on test_a.name  = test_b.name """
q = pd.read_sql(query, conn)
q